<a href="https://colab.research.google.com/github/ljagged/jupyter_notebooks/blob/master/Qwen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🚀 AI Development Environment Setup with Ollama & Tailscale

This notebook configures a complete, secure development environment in Google Colab. It sets up:

1.  **Ollama Server**: Runs large language models like Qwen3.
2.  **Local Models**: Loads models from your Google Drive for persistence and speed.
3.  **Secure Networking**: Uses **Tailscale** to create a private, secure connection between Colab and your local machine. This is more secure and stable than using ngrok.

### Instructions
1.  Set your `TAILSCALE_AUTH_KEY` in the first code cell.
2.  Run the cells in order by selecting `Runtime > Run all`.

In [ ]:
import os

# --- 🔑 CONFIGURATION --- #
# 1. Get a Tailscale auth key from your admin console:
#    https://login.tailscale.com/admin/settings/keys
#    Choose an 'Ephemeral' and 'Reusable' key for best results with Colab.
TAILSCALE_AUTH_KEY = "YOUR_TAILSCALE_AUTH_KEY_HERE" # 🚨 PASTE YOUR KEY HERE

# --- Mount Google Drive & Setup Paths ---
print("🔄 Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive')

DRIVE_ROOT = "/content/drive/MyDrive"
WORKSPACE_PATH = os.path.join(DRIVE_ROOT, "ZedWorkspace")
MODEL_PATH = os.path.join(DRIVE_ROOT, "ollamaModels") # Using your existing folder name
os.makedirs(MODEL_PATH, exist_ok=True)
print(f"✅ Google Drive mounted. Model path set to: {MODEL_PATH}")

### Step 2: Install Dependencies (Ollama & Tailscale)

In [ ]:
%%bash
echo "🔄 Installing Ollama..."
# Install Ollama
curl -fsSL https://ollama.com/install.sh | sh > /dev/null 2>&1
echo "✅ Ollama installed."

echo "🔄 Installing Tailscale..."
# Install Tailscale
curl -fsSL https://tailscale.com/install.sh | sh > /dev/null 2>&1
echo "✅ Tailscale installed."

### Step 3: Connect to Secure Network (Tailscale)

This cell connects your Colab instance to your private Tailscale network, making it securely accessible from your local machine without exposing it to the internet.

In [ ]:
import subprocess
import re
import time

if not TAILSCALE_AUTH_KEY or TAILSCALE_AUTH_KEY == "YOUR_TAILSCALE_AUTH_KEY_HERE":
    print("🚨 ERROR: Tailscale auth key is not set. Please set it in the first cell.")
else:
    print("🚀 Starting Tailscale...")
    # Start tailscale and authenticate
    tailscale_command = f'tailscale up --authkey="{TAILSCALE_AUTH_KEY}" --hostname="colab-ollama-runner" --accept-routes'
    subprocess.run(tailscale_command, shell=True, check=True, capture_output=True)
    print("✅ Tailscale is running.")
    
    # It can take a few seconds for the IP to be assigned
    time.sleep(5)
    
    # Get the Tailscale IP address
    ip_result = subprocess.run(['tailscale', 'ip', '-4'], capture_output=True, text=True)
    tailscale_ip = ip_result.stdout.strip()

    if re.match(r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", tailscale_ip):
        print(f"🔑 Your private Ollama IP is: {tailscale_ip}")
        # This variable will be used later to generate the Zed config
        OLLAMA_PRIVATE_URL = f"http://{tailscale_ip}:11434"
    else:
        print("❌ Could not retrieve Tailscale IP. Please check your Tailscale admin console.")
        OLLAMA_PRIVATE_URL = None

### Step 4: Start Ollama and Load Local Models

In [ ]:
import os
import subprocess
import time

print("🔄 Preparing to start Ollama server...")

# Kill any previous Ollama processes to ensure a clean start
try:
    subprocess.run(["pkill", "-f", "ollama"], check=False)
    print("Ensured previous Ollama processes are stopped.")
    time.sleep(2)
except Exception as e:
    print(f"Could not pkill ollama, continuing: {e}")

# Set up environment for the Ollama subprocess
ollama_env = os.environ.copy()
ollama_env['OLLAMA_HOST'] = "0.0.0.0"  # Listen on all interfaces
ollama_env['OLLAMA_ORIGINS'] = "*"      # Allow all origins (safe within a private Tailscale network)

print(f"Starting Ollama server, listening on {ollama_env['OLLAMA_HOST']}...")

# Start Ollama in the background
ollama_process = subprocess.Popen(
    ["ollama", "serve"],
    env=ollama_env,
    preexec_fn=os.setsid  # Detach from this notebook's session
)

print(f"Ollama server process started with PID: {ollama_process.pid}")
print("Waiting for server to initialize...")
time.sleep(15)

# --- Create Models from Local Filesystem ---
MODELS_TO_CREATE = {
    "nomic-embed-text-local": "nomic-embed-text.mod",
    "qwen3-8b-local": "qwen3_8b.mod",
    "qwen3-30b-local": "qwen3_30b-a3b.mod"
}
print("\n🔄 Creating models from local Modelfiles...")
for model_name, modelfile_name in MODELS_TO_CREATE.items():
    modelfile_path = os.path.join(MODEL_PATH, modelfile_name)
    print(f"Attempting to create '{model_name}' from '{modelfile_path}'...")

    if not os.path.exists(modelfile_path):
        print(f"   ❌ Error: Modelfile not found at {modelfile_path}. Skipping.")
        continue
    try:
        create_command = ["ollama", "create", model_name, "-f", modelfile_path]
        result = subprocess.run(
            create_command,
            capture_output=True, text=True, check=True, timeout=900  # 15-minute timeout for large models
        )
        print(f"   ✅ Successfully created model '{model_name}'.")
    except subprocess.CalledProcessError as e:
        print(f"   ❌ Failed to create model '{model_name}':\n      {e.stderr.strip()}")
    except subprocess.TimeoutExpired:
        print(f"   ❌ Error: Command timed out while creating '{model_name}'.")
    except Exception as e:
        print(f"   ❌ An unexpected error occurred: {e}")

# --- Final Verification --- #
print("\n📊 Verifying available models with 'ollama list'...")
try:
    result = subprocess.run(["ollama", "list"], capture_output=True, text=True, check=True, timeout=20)
    print("✅ Ollama is running with the following models:")
    print(result.stdout)
except Exception as e:
    print(f"❌ Failed to list Ollama models: {e}")

### Step 5: Generate Zed Configuration

This cell generates the `settings.json` content for Zed, using the private Tailscale IP for the connection.

In [ ]:
import json

if 'OLLAMA_PRIVATE_URL' in locals() and OLLAMA_PRIVATE_URL:
    print(f"Generating Zed configuration using private URL: {OLLAMA_PRIVATE_URL}")

    zed_settings = {
        "features": {
            "edit_prediction_provider": "zed"
        },
        "language_models": {
            "ollama": {
                "api_url": OLLAMA_PRIVATE_URL
            }
        },
        "agent": {
            "version": "2",
            "default_model": {
                "provider": "ollama",
                "model": "qwen3-8b-local"
            },
            "inline_assistant_model": {
                "provider": "ollama",
                "model": "qwen3-8b-local"
            },
            "thread_summary_model": {
                "provider": "ollama",
                "model": "qwen3-8b-local"
            }
        }
    }

    # Display the configuration
    print("\n📋 Copy the following JSON into your Zed 'settings.json' file:")
    print("-" * 60)
    print(json.dumps(zed_settings, indent=2))
    print("-" * 60)
else:
    print("❌ Cannot generate Zed config: Tailscale IP was not retrieved successfully.")

### Step 6: Final Instructions & Keep-Alive

Your environment is now running. This final cell will keep the Colab instance active.

In [ ]:
import time

print("="*80)
print("🎉 SETUP COMPLETE! Your secure AI development environment is live.")
print("="*80)

print("\n📋 NEXT STEPS ON YOUR LOCAL COMPUTER:")
print("1. **Install Tailscale**: Download and install it from https://tailscale.com/download")
print("2. **Log In to Tailscale**: Use the same account you used to generate the auth key.")
print("3.  **Check Connection**: Run `tailscale status` in your local terminal. You should see the `colab-ollama-runner` machine listed.")
4.  **Configure Zed**: Copy the JSON configuration from the cell above into your Zed `settings.json` file.
5.  **Restart Zed**: Ensure Zed picks up the new settings.

if 'tailscale_ip' in locals() and tailscale_ip:
    print(f"\nYour private Ollama server is now accessible *only* to you at: http://{tailscale_ip}:11434")
else:
    print("\nCould not determine private Ollama IP. Please check the Tailscale setup cell.")

print("\nThis cell will now run indefinitely to keep the Colab instance and Tailscale connection active.")
print("Press the 'Stop' button on this cell when you are finished to shut down the environment.")

# Keep-alive loop
try:
    while True:
        time.sleep(300) # Sleep for 5 minutes
        print(f"[{time.strftime('%H:%M:%S')}] Keep-alive check. All services running.")
except KeyboardInterrupt:
    print("\n🛑 Shutdown requested. All processes will terminate.")